In [8]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [12]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 'segment_ga=GA1.1.361344545.1732793534; ajs_anonymous_id=34440abb-675b-495f-9e43-91cb2e7a7d1d; _fbp=fb.1.1732793536294.54635627275418320; didomi_token=eyJ1c2VyX2lkIjoiMTkzNzI4YjAtOTJmMi02OTI3LWExZmYtODYxYWRkYTMxOWNmIiwiY3JlYXRlZCI6IjIwMjQtMTEtMjhUMTE6MzI6MTQuNzY3WiIsInVwZGF0ZWQiOiIyMDI0LTExLTI4VDExOjMyOjU3LjU5NVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQIzPkAQIzPkAAHABBENBRFsAP_gAEPgAAiQKoNX_G__bWlr8X73aftkeY1P9_h77sQxBhfJE-4FzLvW_JwXx2ExNA36tqIKmRIAu3TBIQNlGJDURVCgaogVryDMaEyUoTNKJ6BkiFMRM2dYCFxvm4tj-QCY5vr991dx2B-t7dr83dzyy4xHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F_pE2_eT1l_tWvp7D9-cts__XW89_fff_9Pn_-uB_-_3_vf_BU4Akw0KiAMsCQkINAwggQAqCsICKBAEAACQNEBACYMCnYGAC6wkQAgBQADBACAAEGQAIAAAIAEIgAgAKBAABAIFAAGABAMBAAQMAAIALAQCAAEB0DFMCCAQLABIzIiFMCEIBIICWyoQSAIEFcIQizwCIBETBQAAAAAFIAAgLBYHEkgJUJBAFxBtAAAQAIBBAAUIJOTAAEAZstQeDBtGVpgGD5gkQ0wDIAiCMgAAAAA.f_wACHwAAAAA; _gcl_au=1.1.251834761.1732793578; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; kanirante-adv-aauid=b44dfcb4-fe6e-4637-ba61-398b03cb82d5; adit-xandr-id=2992351650503037385; _lr_env_src_ats=false; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20056%7CMCMID%7C43568057574195456610887908598906811472%7CMCAAMLH-1733398378%7C6%7CMCAAMB-1733398378%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1732800778s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.6.0; sui_1pc=1732793578531314C860FBC8A96C504B2AA544D19F95009F0E9EA777; __gsas=ID=d2fca6dfbd891a2a:T=1732793578:RT=1732793578:S=ALNI_MabelxP4kEC1DqY8IXTfYzMine1Og; _lr_sampling_rate=100; ShowMap=false; _csrf=JDlv60TxoZGpyY0PmaJljD8D4xZzvMIJgQ24SJVeYc5n1Ynvf/Jw8dN4CePf2ozCwIsyR7AWkuURlr7QbPfA/reR81WitLZ+TbJz+qaD7bg=; cu=es-es; ASP.NET_SessionId=kjx0ijzc2kpaykud3qlrjpxt; auth=kjx0ijzc2kpaykud3qlrjpxt; ab.storage.deviceId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3A86822635-4043-47e1-5644-5f21ad3cad76%7Ce%3Aundefined%7Cc%3A1732793535694%7Cl%3A1732805275502; _lr_retry_request=true; _hjSession_89611=eyJpZCI6IjYwOGRjYTkxLTJiZmMtNDIyNy05YmRiLTc1NjlkZDI5ZWM2MSIsImMiOjE3MzI4MDYyODc3MTcsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MH0=; reese84=3:UPecnXZcjacv79c+jAK5FQ==:k2dgItfn10zmGQtx7x69xVcCptEZPcmiUvpoDwkhHgMbDxAb4UdnnI4q0GxPXJl3iOOdY6x5MxvmOqkJHQu8hUnwsLrgo5P9m1kxUuEgzBVgcM+sYaz4lwVG/YAD5HDLj9BUM51YI0fx45xzPwka4mZ46lYfy1V5N2mCTLfww4HB6QXsFU6dfD11pLkHGKyDi8d0ySA/50NiS4Z72QPTt2hXOChw7L4g+6iFKY+PqxE9XmdwjyKuC3xMsMgEq0x3b3XdKCCVMKJ9KOuCOdPNjbRvxFZ40RlxkBsM8T2kwr5IV1r+doCuSv4AZbVE9IIUTFL6OeEQmYkI9Xx4uEFjCYRhrIyXckDmbA/wGSLN08gUm+5XV263CJ1vtrfuX+O/1J067lnuZpVgt+hMAa2Xk/xMytbrJqcuMASUFYfPZcgVBQmtAaFdFbbtudtyF2UJ6VT/jtW8NQ7dzJLJQ+uuPU74rrDJV1Fhd+l8LCR1S+o=:vbxp53AT9FEEpIG+3dh3biCFkkER1qZrrFo4dnuGF40=; re_uuid=b72819ca-3639-447e-9e97-3ad959f304a2; _hjSessionUser_89611=eyJpZCI6IjdkOGViNTRiLTFlY2UtNTEyZC04YTM5LWUxODA2OTVkODMzNSIsImNyZWF0ZWQiOjE3MzI4MDYyODc3MTUsImV4aXN0aW5nIjp0cnVlfQ==; __gads=ID=770edfc2b3aafc93:T=1732793579:RT=1732807282:S=ALNI_MY2gogBo_11Qdm7OM-oNFNqhsK9wA; __gpi=UID=00000fa19eb998b1:T=1732793579:RT=1732807282:S=ALNI_MYXR-AAulPyNY35ZiNLwt0W4L7t9g; __eoi=ID=9d729ad46a5d0430:T=1732793579:RT=1732807282:S=AA-Afjbi2NjB7Myn4UQvWLR0_nPj; usunico=28/11/2024:15-11776728; segment_ga_WLSRR9KS1C=GS1.1.1732805351.2.1.1732807323.0.0.0; ab.storage.sessionId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3A110de466-0382-e6f4-9b17-8d132c5aa169%7Ce%3A1732809123740%7Cc%3A1732805275500%7Cl%3A1732807323740; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22Qy9KttgOGoqDuw03SOMI%22%2C%22expiryDate%22%3A%222025-11-28T15%3A22%3A03.787Z%22%7D; cto_bundle=9VYDyF9xOVVFeTRUWERlREw4anh0clVTYkthdnhZUzRXbkNYU3lQN2d6ayUyQmI2YWN3bUljVzlZOFloT3JjVTZHMENzSnNCazFpSGtINkxZTzJpSSUyRjZFRGJ5bld5RXc0Uzh2SWNTSlpmSGRWd3FBcm0zSjhHdXpuR0ZkN2QlMkZZdmY3RTE3eDh5bDdOSE1RaXlxdlN6cklhSXV3UFElM0QlM0Q; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222025-11-28T15%3A22%3A04.649Z%22%7D',
    "Referer": "https://www.idealista.com/alquiler-viviendas/murcia-murcia/",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
}

In [17]:
# Crear una sesión
session = requests.Session()
session.headers.update(headers)

# URL base de Fotocasa
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"

# Realizar la solicitud a la página principal
response = session.get(base_url)
response.raise_for_status()  # Verificar que la solicitud sea exitosa

# Analizar el contenido HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontrar todas las tarjetas de inmuebles
listings = soup.find_all("article", class_="re-CardPackPremium")

# Procesar cada listado
for listing in listings:
    try:
        # Encontrar el enlace al inmueble
        link_tag = listing.find('a', class_='re-CardPackPremium-carousel')
        if not link_tag:
            continue
        href = link_tag.get('href').get_text()
        full_link = f"https://www.fotocasa.es{href}" if href.startswith("/") else f"https://www.fotocasa.es/{href}"

        # Encontrar el título del inmueble
        titulo = listing.find("span", class_="re-I18nPropertyTitle").get_text()

        # Sacar el precio
        precio = listing.find('span', class_="re-CardPrice").get_text(strip=True)

        # Extraer características principales (habitaciones, baños, superficie)
        features = listing.find("ul", class_="re-CardFeaturesWithIcons-wrapper")
        habitaciones = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--rooms").get_text(strip=True) if features else "N/A"
        banos = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--bathrooms").get_text(strip=True) if features else "N/A"
        superficie = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--surface").get_text(strip=True) if features else "N/A"
        planta = features.find('span', class_='re-CardFeaturesWithIcons-feature-icon--floor').get_text(strip=True) if features else "N/A"

        # Mostrar los resultados básicos
        print(f"Título: {titulo}")
        print(f"Precio: {precio}")
        print(f"Habitaciones: {habitaciones}")
        print(f"Baños: {banos}")
        print(f"Superficie: {superficie}")
        print(f"Planta: {planta}")
        print(f"Enlace: {full_link}")

        # Si hay enlace, entrar en la página del inmueble
        if full_link:
            detail_response = session.get(full_link)
            detail_response.raise_for_status()

            # Analizar el contenido HTML de la página del inmueble
            detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

            # Extraer información adicional
            # Por ejemplo, descripción del inmueble
            descripcion = detail_soup.find("p", class_="re-DetailDescription-text")
            descripcion_text = descripcion.get_text(strip=True) if descripcion else "Descripción no disponible"

            # Extraer dirección exacta si está disponible
            direccion = detail_soup.find("span", class_="re-DetailHeader-address")
            direccion_text = direccion.get_text(strip=True) if direccion else "Dirección no disponible"

            # Extraer características detalladas
            detailed_features = detail_soup.find_all('div', class_='re-DetailFeaturesList-feature')
            caracteristicas = []
            for feature in detailed_features:
                label = feature.find('p', class_='re-DetailFeaturesList-featureLabel').get_text(strip=True)
                value = feature.find('div', class_='re-DetailFeaturesList-featureValue').get_text(strip=True)
                caracteristicas.append((label, value))

            # Mostrar la información adicional
            print("Información adicional:")
            print(f"Descripción: {descripcion_text}")
            print(f"Dirección: {direccion_text}")
            print("Características detalladas:")
            for label, value in caracteristicas:
                print(f"{label}: {value}")
            print("=" * 50)

    except Exception as e:
        print(f"Error procesando un listado: {e}")

HTTPError: 404 Client Error: Not Found for url: https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/

In [16]:
print({link_tag})

{<a class="re-CardPackPremium-info-container" href="/es/alquiler/vivienda/cartagena/aire-acondicionado-calefaccion-parking-ascensor/184987623/d?from=list" title="Piso de alquiler en Casco Antiguo"><h3 class="re-CardHeader"><span class="re-CardTitle re-CardTitle--big"><span class="re-I18nPropertyTitle"><strong>Piso</strong> con balcón<!-- --> en Casco Antiguo </span></span><span class="re-CardPriceContainer"><span class="re-CardPriceComposite"><span class="re-CardPrice">800 €</span></span><span class="re-CardTimeAgo re-CardTimeAgo--isInlineWithPrice" data-testid="CardTimeAgo"> Novedad</span></span></h3><ul class="re-CardFeaturesWithIcons-wrapper re-CardFeaturesWithIcons-wrapper--isTwoLines"><li class="re-CardFeaturesWithIcons-feature"><span class="re-CardFeaturesWithIcons-feature-icon re-CardFeaturesWithIcons-feature-icon--rooms">2 habs.</span></li><li class="re-CardFeaturesWithIcons-feature"><span class="re-CardFeaturesWithIcons-feature-icon re-CardFeaturesWithIcons-feature-icon--bathr

In [18]:
print({href})

{'/es/alquiler/vivienda/cartagena/aire-acondicionado-calefaccion-parking-ascensor/184987623/d?from=list'}
